In [1]:
#!pip -q install git+https://github.com/huggingface/transformers

In [2]:
#!pip install tiktoken chromadb langchain

In [3]:
# from git import Repo
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language

In [4]:
# Load
loader = GenericLoader.from_filesystem(
    "/home/ec2-user/SageMaker/VegRD_APD_Fruit_Phenotyping/scripts",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)
documents = loader.load()
len(documents)

130

In [5]:
len(documents)

130

In [6]:
documents[0]

Document(page_content='import numpy as np\nimport os, sys\nfrom collections import defaultdict\nfrom get_data import get_images\nimport re\n\ndef update_config(config_defaults,**kwargs):\n    new_config = config_defaults.copy()\n    for key, value in kwargs.items():\n        if key in new_config:\n            new_config[key] = value\n        else:\n            raise KeyError(f"Unknown configuration key: {key}")\n    return new_config\n\n\ndef pattern_exists_in_string(string, pattern="BLK"):\n    if re.search(pattern, string):\n        return True\n    else:\n        return False\n\n\n# A dictionary of projects and images\nprojs = defaultdict(list)\n\nwith open("new_images.txt", "r") as f:\n    for l in f.readlines():\n        l=l.strip()\n        splitted= l.split("/")\n        projs[splitted[1]].append(l)\n\nfor k,v in projs.items():\n    get_images(k,s3_bucket="model-deployment-automation", files_to_copy=v)\n    if pattern_exists_in_string(k,pattern="BLK"):\n        from config impor

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

254

In [8]:
#!pip install langchain sentence_transformers

In [9]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
embeddings = HuggingFaceInstructEmbeddings(
    query_instruction="Query about the code base: ",
    model_kwargs = {'device': 'cpu'}
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


max_seq_length  512


In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [11]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory

In [19]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
#model = model.half()
#model=model.to("cuda")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000)
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
# To check the device and dtype
# Get the first parameter of the model and check its device and dtype
first_param = next(model.parameters())
model_device = first_param.device
model_dtype = first_param.dtype

print(f"Model is on device: {model_device}")
print(f"Model data type: {model_dtype}")

Model is on device: cpu
Model data type: torch.float32


In [22]:
## Test LLM without Context

from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 First, we need to understand what the brain is. The brain is a complex organ that controls all the functions of the human body. It is responsible for thinking, feeling, moving, and even breathing.

Now, imagine if we could measure the electrical activity of the brain while it is functioning. This would give us a better understanding of how the brain works and how it processes information. This is where electroencephalography (EEG) comes in.

EEG is a technique used to measure the electrical activity of the brain using electrodes placed on the scalp. These electrodes pick up tiny electrical signals produced by the brain cells, which are then recorded and analyzed. By studying these signals, we can learn more about the brain's functions and how it processes information.

EEG is a non-invasive procedure, which means it does not involve any surgery or needles. It is a safe and effective way to study the brain and its functions. EEG is used in a variety of settings, including hospitals, re

In [23]:
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "How can I initialize a ReAct agent?"
result = qa(question)
result["answer"]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
questions = [
    "How is curved backbone of hot peppers calculated?",
    "How to turn on/off visulaizations of phenotypes?",
    "How to turn on/off visulaizations of detections and phenotypes of hot peppers lsit all methods?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")